In [2]:
import os
import logging
from langchain.text_splitter import CharacterTextSplitter
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_core.documents import Document
from google.cloud import documentai_v1beta3 as documentai
from qdrant_client import QdrantClient, models
from settings import env_settings
from dotenv import load_dotenv

load_dotenv()


logger = logging.getLogger(__name__)

QDRANT_URL = env_settings.QDRANT_URL
QDRANT_API_KEY = env_settings.QDRANT_API_KEY
OPENAI_API_KEY = env_settings.OPENAI_API_KEY
GCP_PROJECT_ID = env_settings.GCP_PROJECT_ID
GCP_LOCATION = env_settings.GCP_LOCATION
GCP_PROCESSOR_ID = env_settings.GCP_PROCESSOR_ID
GCP_PROCESSOR_VERSION = env_settings.GCP_PROCESSOR_VERSION

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = env_settings.GOOGLE_APPLICATION_CREDENTIALS


class EthanRAG:
    _qdrant_client: QdrantClient = None

    def __init__(self):
        if EthanRAG._qdrant_client is None:
            EthanRAG._qdrant_client = self.connect_qdrant()

    @staticmethod
    def connect_qdrant() -> QdrantClient:
        return QdrantClient(
            api_key=QDRANT_API_KEY,
            url=QDRANT_URL,
        )

    @staticmethod
    async def extract_text_from_pdf(file_path: str) -> str:
        try:
            logger.info(f"Processing with Google Document AI: {file_path}")
            client = documentai.DocumentProcessorServiceClient()
            name = client.processor_version_path(
                GCP_PROJECT_ID, GCP_LOCATION, GCP_PROCESSOR_ID, GCP_PROCESSOR_VERSION
            )
            with open(file_path, "rb") as pdf_file:
                raw_document = documentai.RawDocument(
                    content=pdf_file.read(), mime_type="application/pdf"
                )
            request = documentai.ProcessRequest(
                name=name,
                raw_document=raw_document,
                process_options=documentai.ProcessOptions(
                    ocr_config=documentai.OcrConfig(
                        enable_native_pdf_parsing=True)
                ),
            )
            result = client.process_document(request=request)
            document = result.document
            extracted_text = document.text
            return extracted_text
        except Exception as e:
            logger.error(f"Error extracting text from PDF: {e}")
            raise e

k:\EthanAI\ethan-ai-rag\env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
ethan_rag = EthanRAG()

text = await ethan_rag.extract_text_from_pdf(
    file_path="K:\\EthanAI\\ethan-ai-rag\\uploads\\bala_monthly_report_dec_24-1-5.pdf",
)

In [9]:
from llama_index.core.node_parser import SentenceWindowNodeParser
from llama_index.core import Document

In [10]:
documents = [Document(text=text, metadata={"filename": "bala_monthly_report_dec_24-1-5.pdf"})]

In [6]:
node_parser = SentenceWindowNodeParser.from_defaults(window_size=3, window_metadata_key="window")

In [11]:
nodes = node_parser.get_nodes_from_documents(documents)

In [47]:
final_response = """
Final Investment Recommendation
Based on the comprehensive analysis of economic indicators, stock market performance, and overall economic stability, here is a summary of the investment potential for India, the USA, and China:
India

    Strengths: Highest GDP growth rate at 7.58%, indicating strong economic expansion and potential for high returns.
    Challenges: High inflation rate at 6.95%, which could impact purchasing power and consumer spending.
    Stock Market: Positive growth trajectory driven by infrastructure and private investment.

USA

    Strengths: Moderate inflation rate at 4.12% and the lowest unemployment rate at 3.63%, indicating a stable economic environment and strong labor market.
    Challenges: Lower GDP growth rate at 2.54% compared to India and China.
    Stock Market: Robust performance, particularly in technology, reflecting economic resilience.

China

    Strengths: Low inflation rate at 0.23%, suggesting a stable cost of living and strong purchasing power.
    Challenges: Slightly higher unemployment rate at 5.2%, indicating some labor market challenges.
    Stock Market: Signs of recovery and growth, with potential to outperform U.S. markets.

Conclusion

    India: Offers the highest growth potential but faces inflation challenges. Suitable for investors seeking high returns and willing to manage inflation risks.
    USA: Provides a stable investment environment with moderate inflation and a strong labor market. Ideal for investors seeking stability and consistent returns.
    China: Presents a stable economic environment with low inflation and promising stock market recovery. Suitable for investors looking for stability and growth potential.

Recommendation

    For High Growth Potential: Consider investing in India for its rapid economic expansion.
    For Stability and Consistent Returns: The USA offers a balanced investment environment.
    For Stability with Growth Potential: China provides a stable economic setting with promising market recovery.

Investors should consider their risk tolerance, investment goals, and market conditions when making a decision. Diversifying investments across these markets could also be a strategic approach to balance risk and returns.
"""

In [48]:
summary_prompt = f"""
You are a podcast host summarizing a recent analysis. Create a concise, engaging summary (2-3 sentences) 
of the following content for a podcast audience. Use a friendly and conversational tone, 
and avoid technical jargon unless necessary also add some anecdotes or examples to make it funny and relatable: 

{final_response}

Summary:
"""

In [49]:
from langchain_openai import ChatOpenAI
from settings import env_settings

In [50]:
llm = ChatOpenAI(
    model="gpt-4o",
    api_key=env_settings.OPENAI_API_KEY
)

In [51]:
summary_response = await llm.ainvoke(summary_prompt)
summary_text = summary_response.content

In [52]:
from gtts import gTTS
from google.cloud import texttospeech

In [53]:
client = texttospeech.TextToSpeechClient()
synthesis_input = texttospeech.SynthesisInput(text=summary_text)
voice = texttospeech.VoiceSelectionParams(
    language_code="en-US", ssml_gender=texttospeech.SsmlVoiceGender.NEUTRAL, name="en-US-Chirp3-HD-Charon",
)
audio_config = texttospeech.AudioConfig(
    audio_encoding=texttospeech.AudioEncoding.MP3
)
response = client.synthesize_speech(
    input=synthesis_input, voice=voice, audio_config=audio_config
)
with open("output.mp3", "wb") as out:
    out.write(response.audio_content)
    print('Audio content written to file "output.mp3"')

Audio content written to file "output.mp3"
